In [1]:
import pandas as pd
import datetime as dt
import hvplot.pandas

In [2]:
import datetime as dt

# Get today's date as a Pandas datetime object
first_day = '7/14/2021'
days = (pd.to_datetime(dt.datetime.today().strftime('%Y-%m-%d')) - pd.to_datetime(first_day)).days
days

921

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
coin_list = cg.get_coins_list()

coins = pd.DataFrame(coin_list)

coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0-knowledge-network,0kn,0 Knowledge Network
3,0-mee,ome,O-MEE
4,0vix-protocol,vix,0VIX Protocol
...,...,...,...
12349,zynecoin,zyn,Zynecoin
12350,zynergy,zyn,Zynergy
12351,zyrri,zyr,Zyrri
12352,zyx,zyx,ZYX


In [5]:
tokens = [
    'GTC',
    'GIV',
    'ENS',
    'KLIMA',
    'RAD',
    'BRIGHT',
    'HONEY',
    'TEC',
    'SEED',
    'PAN',
]

In [6]:
coins = coins[coins['symbol'].str.contains('|'.join(tokens).lower())]
coins

,id,symbol,name
183,aave-v3-ens,aens,Aave v3 ENS
864,astra-dao-2,astradao,Astra DAO
958,ausd-seed-acala,aseed,aUSD SEED (Acala)
959,ausd-seed-karura,aseed,aUSD SEED (Karura)
1382,bigpanda,bigpanda,BIGPANDA
...,...,...,...
9976,spankchain,spank,SpankChain
10544,tenshi,tenshi,Tenshi
10847,toxicgarden-finance-seed,seed,ToxicGarden.finance SEED
10872,tradex-ai,tradex,TradeX AI


In [7]:
token_ids = [
    'bright-token',
    # 'ethereum-name-service',
    'gitcoin',
    'giveth',
    'rad',
    'honey-finance',
    # 'token-engineering-commons',
    'metagame',
    
]

exact_match_regex = ['^'+id+'$' for id in token_ids]

In [8]:
coins[coins['id'].str.match('|'.join(exact_match_regex).lower())]

,id,symbol,name
1918,bright-token,bright,BrightID
4662,gitcoin,gtc,Gitcoin
4666,giveth,giv,Giveth
5230,honey-finance,honey,Honey Finance
6841,metagame,seed,MetaGame
8845,rad,rad,RAD


In [9]:
token_id = 'bright-token'
quote = 'ETH'
days = 730

# Pull prices from coin gecko
prices = cg.get_coin_market_chart_by_id(token_id, quote, days)

# Little bit of clean up
prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
prices = {a:[i[1] for i in b] for a,b in prices.items()}

# Make a dataframe time series
prices_df = pd.DataFrame(prices)

prices_df['token_id'] = token_id

prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000115,0.0,77.759766,1642896000000,bright-token
1,0.000113,0.0,77.128195,1642982400000,bright-token
2,0.000121,0.0,52.110867,1643068800000,bright-token
3,0.000118,0.0,73.638198,1643155200000,bright-token
4,0.000121,0.0,107.827147,1643241600000,bright-token
...,...,...,...,...,...
726,0.000015,0.0,0.219993,1705622400000,bright-token
727,0.000015,0.0,0.108248,1705708800000,bright-token
728,0.000015,0.0,0.026390,1705795200000,bright-token
729,0.000015,0.0,0.075399,1705881600000,bright-token


In [10]:
def get_price_history(token_id: str, quote: str, days: int) -> pd.DataFrame:
    # Pull prices from coin gecko
    prices = cg.get_coin_market_chart_by_id(token_id, quote, days)
    
    # Little bit of clean up
    prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
    prices = {a:[i[1] for i in b] for a,b in prices.items()}
    
    # Make a dataframe time series
    prices_df = pd.DataFrame(prices)
    
    prices_df['token_id'] = token_id
    
    return prices_df

In [11]:
token_ids

['bright-token', 'gitcoin', 'giveth', 'rad', 'honey-finance', 'metagame']

In [12]:
df = get_price_history(token_ids[1], quote='ETH', days=days)
df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.002886,41062.021218,16309.009276,1642896000000,gitcoin
1,0.002869,40788.880351,10703.837297,1642982400000,gitcoin
2,0.002739,38962.392328,12202.974835,1643068800000,gitcoin
3,0.002814,40067.931110,8473.806503,1643155200000,gitcoin
4,0.002830,40138.149130,8148.328007,1643241600000,gitcoin
...,...,...,...,...,...
726,0.000455,27692.924515,2312.055546,1705622400000,gitcoin
727,0.000446,27145.922481,2784.625980,1705708800000,gitcoin
728,0.000501,30504.383874,20686.134577,1705795200000,gitcoin
729,0.000515,31348.823922,8805.990499,1705881600000,gitcoin


In [13]:
tec_data = pd.read_csv('data/TEC-prices.csv')

tec_data['timestamp'] = pd.to_datetime(tec_data['Date (00:00 UTC,Coingecko)']).dt.normalize()

tec_data['prices'] = tec_data['TEC price (ETH)']
tec_data['token_id'] = 'token-engineering-commons'
tec_data['market_caps'] = 0
tec_data['total_volumes'] = 0

tec_data = tec_data[prices_df[prices_df['token_id']=='token-engineering-commons'].columns]

tec_data

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000514,0,0,2021-07-14,token-engineering-commons
1,0.000501,0,0,2021-07-15,token-engineering-commons
2,0.000523,0,0,2021-07-16,token-engineering-commons
3,0.000534,0,0,2021-07-17,token-engineering-commons
4,0.000526,0,0,2021-07-18,token-engineering-commons
...,...,...,...,...,...
724,0.000269,0,0,2023-07-08,token-engineering-commons
725,0.000266,0,0,2023-07-09,token-engineering-commons
726,0.000271,0,0,2023-07-10,token-engineering-commons
727,0.000265,0,0,2023-07-11,token-engineering-commons


In [14]:
data = []
for token_id in token_ids:
    print(f"Fetching data for token_id: {token_id}")
    df = get_price_history(token_id, quote='ETH', days=days)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.normalize()

    if token_id == 'token-engineering-commons':
        df = df.set_index('timestamp').combine_first(tec_data.set_index('timestamp')).reset_index()
    data.append(df)

print(f"Fetching data for ETH/USD")
df = get_price_history('ethereum', quote='usd', days=days)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.normalize()
data.append(df)

Fetching data for token_id: bright-token
Fetching data for token_id: gitcoin
Fetching data for token_id: giveth
Fetching data for token_id: rad
Fetching data for token_id: honey-finance
Fetching data for token_id: metagame
Fetching data for ETH/USD


In [15]:
# Concatenate all of the data into one dataframe
prices_df = pd.concat(data)
prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000115,0.000000e+00,7.775977e+01,2022-01-23,bright-token
1,0.000113,0.000000e+00,7.712820e+01,2022-01-24,bright-token
2,0.000121,0.000000e+00,5.211087e+01,2022-01-25,bright-token
3,0.000118,0.000000e+00,7.363820e+01,2022-01-26,bright-token
4,0.000121,0.000000e+00,1.078271e+02,2022-01-27,bright-token
...,...,...,...,...,...
726,2467.328979,2.965593e+11,1.509846e+10,2024-01-19,ethereum
727,2489.618065,2.992828e+11,1.351457e+10,2024-01-20,ethereum
728,2470.263615,2.968603e+11,4.996301e+09,2024-01-21,ethereum
729,2454.909543,2.952255e+11,4.334793e+09,2024-01-22,ethereum


In [71]:
prices_df = pd.concat([prices_df, tec_data])

In [85]:
# prices_df.to_csv('data/public_goods_prices.csv', index=False)

In [72]:
prices_df.dtypes

prices                      float64
market_caps                 float64
total_volumes               float64
timestamp            datetime64[ns]
token_id                     object
normalized_prices           float64
dtype: object

In [74]:
prices_df[prices_df['token_id']=='token-engineering-commons'].hvplot.step(x='timestamp', y='prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

In [75]:
import hvplot.pandas

In [76]:
prices_df.hvplot.step(x='timestamp', y='prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

Create normalized prices.

In [77]:
prices_df['normalized_prices'] = prices_df['prices'] / prices_df.groupby('token_id')['prices'].transform('first')

In [78]:
prices_df.hvplot.step(x='timestamp', y='normalized_prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (normalized_prices)

Giveth has some anomolous behavior near the beginning. I'm going to remove the first 20 days of giveth.

In [79]:
import holoviews as hv

prices_df_adjusted = prices_df.copy(deep=True)

prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'] = prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'].iloc[20:]

prices_df_adjusted['normalized_prices'] = prices_df_adjusted['prices'] / prices_df_adjusted.groupby('token_id')['prices'].transform('first')

In [80]:
import datetime as dt

In [81]:
d = dt.date.today()

In [82]:
import datetime

def ordinal(n):
    """Return number with ordinal suffix."""
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"

def formatted_date(past=False):
    """Return today's date in the desired format."""
    today = datetime.date.today()
    month = today.strftime('%b')
    day = ordinal(today.day)
    year = today.year
    if past:
        year = year - 2
    return f"{month} {day} {year}"

print(formatted_date())

Jan 21st 2024


In [83]:
today = formatted_date()
past = formatted_date(past=True)

In [84]:
plot = prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].hvplot.line(logy=False, legend='top_right', grid=True, rot=45, title=f'Public Goods Tokens Normalized against Ethereum {past} - {today}',x='timestamp', y='normalized_prices', by='token_id', width=1400, height=600, line_width=3) * hv.HLine(1)
plot

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [28]:
plot.opts(xticks=[(date, date.strftime('%B %Y')) for date in pd.date_range(start=prices_df_adjusted['timestamp'].min(), end=prices_df_adjusted['timestamp'].max(), freq='MS')])

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [29]:
prices_df_adjusted['returns'] = prices_df_adjusted.groupby('token_id')['normalized_prices'].transform(lambda x: x.bfill().pct_change()+1)

In [30]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000115,0.000000e+00,7.775977e+01,2022-01-23,bright-token,1.000000,NaN
1,0.000113,0.000000e+00,7.712820e+01,2022-01-24,bright-token,0.986009,0.986009
2,0.000121,0.000000e+00,5.211087e+01,2022-01-25,bright-token,1.055293,1.070267
3,0.000118,0.000000e+00,7.363820e+01,2022-01-26,bright-token,1.025065,0.971356
4,0.000121,0.000000e+00,1.078271e+02,2022-01-27,bright-token,1.055378,1.029572
...,...,...,...,...,...,...,...
726,2467.328979,2.965593e+11,1.509846e+10,2024-01-19,ethereum,1.024903,0.975873
727,2489.618065,2.992828e+11,1.351457e+10,2024-01-20,ethereum,1.034162,1.009034
728,2470.263615,2.968603e+11,4.996301e+09,2024-01-21,ethereum,1.026122,0.992226
729,2454.909543,2.952255e+11,4.334793e+09,2024-01-22,ethereum,1.019744,0.993784


In [31]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
giveth,0.999817,0.156049,-0.001172
metagame,0.998914,0.069461,-0.015637
gitcoin,0.998698,0.047053,-0.027671
bright-token,0.998276,0.046252,-0.037276
rad,0.995049,0.098520,-0.050250
honey-finance,0.997106,0.055895,-0.051777


In [32]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').apply(lambda x: x['normalized_prices'].iloc[-1] / x['returns'].std()).sort_values(ascending=False)

token_id
gitcoin          3.768361
metagame         3.550135
bright-token     2.796049
honey-finance    1.121110
giveth           0.167774
rad              0.022392
dtype: float64

In [33]:
prices_df_adjusted.groupby('token_id')['normalized_prices'].apply(lambda x: (x.iloc[-1] / x.std())).sort_values(ascending=False)

token_id
ethereum         4.600092
metagame         0.827969
gitcoin          0.806406
bright-token     0.539641
honey-finance    0.133491
giveth           0.086642
rad              0.007521
Name: normalized_prices, dtype: float64

In [34]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000115,0.000000e+00,7.775977e+01,2022-01-23,bright-token,1.000000,NaN
1,0.000113,0.000000e+00,7.712820e+01,2022-01-24,bright-token,0.986009,0.986009
2,0.000121,0.000000e+00,5.211087e+01,2022-01-25,bright-token,1.055293,1.070267
3,0.000118,0.000000e+00,7.363820e+01,2022-01-26,bright-token,1.025065,0.971356
4,0.000121,0.000000e+00,1.078271e+02,2022-01-27,bright-token,1.055378,1.029572
...,...,...,...,...,...,...,...
726,2467.328979,2.965593e+11,1.509846e+10,2024-01-19,ethereum,1.024903,0.975873
727,2489.618065,2.992828e+11,1.351457e+10,2024-01-20,ethereum,1.034162,1.009034
728,2470.263615,2.968603e+11,4.996301e+09,2024-01-21,ethereum,1.026122,0.992226
729,2454.909543,2.952255e+11,4.334793e+09,2024-01-22,ethereum,1.019744,0.993784


In [35]:
prices_df_adjusted.reset_index()['returns'].bfill().idxmax()

1596

In [36]:
prices_df_adjusted.iloc[4484]

prices                       1626.436341
market_caps          195463813946.391968
total_volumes          8399894536.861646
timestamp            2023-09-15 00:00:00
token_id                        ethereum
normalized_prices               0.675605
returns                         1.011631
Name: 600, dtype: object

In [37]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000115,0.000000e+00,7.775977e+01,2022-01-23,bright-token,1.000000,NaN
1,0.000113,0.000000e+00,7.712820e+01,2022-01-24,bright-token,0.986009,0.986009
2,0.000121,0.000000e+00,5.211087e+01,2022-01-25,bright-token,1.055293,1.070267
3,0.000118,0.000000e+00,7.363820e+01,2022-01-26,bright-token,1.025065,0.971356
4,0.000121,0.000000e+00,1.078271e+02,2022-01-27,bright-token,1.055378,1.029572
...,...,...,...,...,...,...,...
726,2467.328979,2.965593e+11,1.509846e+10,2024-01-19,ethereum,1.024903,0.975873
727,2489.618065,2.992828e+11,1.351457e+10,2024-01-20,ethereum,1.034162,1.009034
728,2470.263615,2.968603e+11,4.996301e+09,2024-01-21,ethereum,1.026122,0.992226
729,2454.909543,2.952255e+11,4.334793e+09,2024-01-22,ethereum,1.019744,0.993784


In [38]:
prices_df_adjusted[prices_df_adjusted['token_id']=='token-engineering-commons']

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns


In [39]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last').sort_values(ascending=False).to_frame(name=f'Return {past} - {today}').reset_index()

,token_id,Return Jan 21st 2022 - Jan 21st 2024
0,metagame,0.246596
1,gitcoin,0.177311
2,bright-token,0.129324
3,honey-finance,0.062665
4,giveth,0.026181
5,rad,0.002206


In [40]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std').sort_values(ascending=True).to_frame(name=f'Volatility {past} - {today}').reset_index()

,token_id,Volatility Jan 21st 2022 - Jan 21st 2024
0,bright-token,0.046252
1,gitcoin,0.047053
2,honey-finance,0.055895
3,metagame,0.069461
4,rad,0.098520
5,giveth,0.156049


In [41]:
(prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last') / prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std')).sort_values(ascending=False).to_frame(name=f'Pseudo Sharpe {past} - {today}').reset_index()

,token_id,Pseudo Sharpe Jan 21st 2022 - Jan 21st 2024
0,gitcoin,3.768361
1,metagame,3.550135
2,bright-token,2.796049
3,honey-finance,1.121110
4,giveth,0.167774
5,rad,0.022392


In [42]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
giveth,0.999817,0.156049,-0.001172
metagame,0.998914,0.069461,-0.015637
gitcoin,0.998698,0.047053,-0.027671
bright-token,0.998276,0.046252,-0.037276
rad,0.995049,0.098520,-0.050250
honey-finance,0.997106,0.055895,-0.051777


In [43]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000115,0.000000e+00,7.775977e+01,2022-01-23,bright-token,1.000000,NaN
1,0.000113,0.000000e+00,7.712820e+01,2022-01-24,bright-token,0.986009,0.986009
2,0.000121,0.000000e+00,5.211087e+01,2022-01-25,bright-token,1.055293,1.070267
3,0.000118,0.000000e+00,7.363820e+01,2022-01-26,bright-token,1.025065,0.971356
4,0.000121,0.000000e+00,1.078271e+02,2022-01-27,bright-token,1.055378,1.029572
...,...,...,...,...,...,...,...
726,2467.328979,2.965593e+11,1.509846e+10,2024-01-19,ethereum,1.024903,0.975873
727,2489.618065,2.992828e+11,1.351457e+10,2024-01-20,ethereum,1.034162,1.009034
728,2470.263615,2.968603e+11,4.996301e+09,2024-01-21,ethereum,1.026122,0.992226
729,2454.909543,2.952255e+11,4.334793e+09,2024-01-22,ethereum,1.019744,0.993784


In [44]:
prices_df_adjusted['returns'].rolling(7).std()

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
726    0.028647
727    0.026332
728    0.024487
729    0.020033
730    0.018733
Name: returns, Length: 4615, dtype: float64

In [45]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000115,0.000000e+00,7.775977e+01,2022-01-23,bright-token,1.000000,NaN
1,0.000113,0.000000e+00,7.712820e+01,2022-01-24,bright-token,0.986009,0.986009
2,0.000121,0.000000e+00,5.211087e+01,2022-01-25,bright-token,1.055293,1.070267
3,0.000118,0.000000e+00,7.363820e+01,2022-01-26,bright-token,1.025065,0.971356
4,0.000121,0.000000e+00,1.078271e+02,2022-01-27,bright-token,1.055378,1.029572
...,...,...,...,...,...,...,...
726,2467.328979,2.965593e+11,1.509846e+10,2024-01-19,ethereum,1.024903,0.975873
727,2489.618065,2.992828e+11,1.351457e+10,2024-01-20,ethereum,1.034162,1.009034
728,2470.263615,2.968603e+11,4.996301e+09,2024-01-21,ethereum,1.026122,0.992226
729,2454.909543,2.952255e+11,4.334793e+09,2024-01-22,ethereum,1.019744,0.993784


In [46]:
prices_df_adjusted_tokens = prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].copy(deep=True)

In [47]:
prices_df_adjusted_tokens['volatility'] = prices_df_adjusted_tokens.groupby('token_id')['returns'].transform(
    lambda x: x.rolling(90).std().bfill()
)

In [48]:
prices_df_adjusted_tokens

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns,volatility
0,0.000115,0.0,77.759766,2022-01-23,bright-token,1.000000,NaN,0.034156
1,0.000113,0.0,77.128195,2022-01-24,bright-token,0.986009,0.986009,0.034156
2,0.000121,0.0,52.110867,2022-01-25,bright-token,1.055293,1.070267,0.034156
3,0.000118,0.0,73.638198,2022-01-26,bright-token,1.025065,0.971356,0.034156
4,0.000121,0.0,107.827147,2022-01-27,bright-token,1.055378,1.029572,0.034156
...,...,...,...,...,...,...,...,...
391,0.000645,0.0,0.391146,2024-01-14,metagame,0.204021,1.000000,0.091165
392,0.000607,0.0,0.039512,2024-01-15,metagame,0.191946,0.940814,0.091282
393,0.000780,0.0,0.011783,2024-01-16,metagame,0.246606,1.284768,0.096055
394,0.000780,0.0,0.011782,2024-01-16,metagame,0.246596,0.999961,0.095720


In [49]:
prices_df_adjusted_tokens.hvplot.line(
    logy=True, 
    legend='top_right', 
    grid=True, rot=45, 
    title=f'Public Goods Tokens 90 Day Volatility Against Ethereum {past} - {today}',
    x='timestamp', y='volatility', by='token_id', width=1400, height=600, line_width=3) 

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (volatility)

In [50]:
prices_df_adjusted_tokens

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns,volatility
0,0.000115,0.0,77.759766,2022-01-23,bright-token,1.000000,NaN,0.034156
1,0.000113,0.0,77.128195,2022-01-24,bright-token,0.986009,0.986009,0.034156
2,0.000121,0.0,52.110867,2022-01-25,bright-token,1.055293,1.070267,0.034156
3,0.000118,0.0,73.638198,2022-01-26,bright-token,1.025065,0.971356,0.034156
4,0.000121,0.0,107.827147,2022-01-27,bright-token,1.055378,1.029572,0.034156
...,...,...,...,...,...,...,...,...
391,0.000645,0.0,0.391146,2024-01-14,metagame,0.204021,1.000000,0.091165
392,0.000607,0.0,0.039512,2024-01-15,metagame,0.191946,0.940814,0.091282
393,0.000780,0.0,0.011783,2024-01-16,metagame,0.246606,1.284768,0.096055
394,0.000780,0.0,0.011782,2024-01-16,metagame,0.246596,0.999961,0.095720


In [51]:
df = prices_df_adjusted_tokens.pivot_table(index='timestamp', columns='token_id', values='returns').resample('D').mean()
df

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad
timestamp,,,,,,
2022-01-24,0.986009,0.993972,NaN,NaN,NaN,0.782832
2022-01-25,1.070267,0.954810,NaN,NaN,NaN,0.995074
2022-01-26,0.971356,1.027173,NaN,NaN,NaN,1.062397
2022-01-27,1.029572,1.005872,NaN,NaN,NaN,1.555456
2022-01-28,0.979013,0.988939,NaN,NaN,NaN,1.356167
...,...,...,...,...,...,...
2024-01-18,1.017760,0.977444,1.013536,NaN,NaN,0.960484
2024-01-19,1.006344,0.951911,0.997261,NaN,NaN,1.022836
2024-01-20,0.987633,0.980488,0.975119,NaN,NaN,1.001421


In [52]:
df.isna().sum()

token_id
bright-token       0
gitcoin            0
giveth            20
honey-finance    117
metagame         335
rad               50
dtype: int64

In [53]:
df.cumprod().bfill().hvplot.line(logy=True)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (value)

In [54]:
cov = df.corr()
cov

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad
token_id,,,,,,
bright-token,1.000000,0.008222,0.027665,0.142036,-0.013348,0.061492
gitcoin,0.008222,1.000000,-0.035642,-0.079764,0.023398,-0.015562
giveth,0.027665,-0.035642,1.000000,0.079334,0.260209,0.017589
honey-finance,0.142036,-0.079764,0.079334,1.000000,0.155669,0.393345
metagame,-0.013348,0.023398,0.260209,0.155669,1.000000,0.009024
rad,0.061492,-0.015562,0.017589,0.393345,0.009024,1.000000


In [55]:
cov.hvplot.heatmap(cmap='reds', colorbar=True, rot=45, width=800, height=500, title="Public Goods Tokens Correlation")

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)


:HeatMap   [columns,index]   (value)

In [56]:
df.mean()

token_id
bright-token     0.998293
gitcoin          0.998682
giveth           0.999808
honey-finance    0.997090
metagame         0.998550
rad              0.995055
dtype: float64

# This section needs to be corrected.

Don't impute. Don't dropna. 

Start with prices, backfill missing data. Then compute returns.

In [57]:
import numpy as np

In [58]:
df_filled = df.where(~df.isna(), np.random.normal(df.mean(), df.std(), size=df.shape))

In [59]:
df_filled = df.dropna()

In [60]:
df_filled.cumprod().hvplot.line(logy=True)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (value)

In [61]:
df_filled

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad
timestamp,,,,,,
2022-04-09,0.949924,0.927148,1.002523,1.041729,1.145661,0.987208
2022-04-10,0.968138,1.038673,0.976691,0.928047,1.000000,0.783287
2022-04-11,0.984979,0.976918,1.060522,1.225002,0.998845,0.910245
2022-04-12,1.110989,0.943426,0.985020,1.091937,1.110623,1.009557
2022-04-13,0.889215,1.046814,0.937338,0.955503,0.999818,0.936462
...,...,...,...,...,...,...
2023-12-09,0.986881,1.026950,0.999784,0.720835,0.998093,0.991985
2023-12-10,1.007872,0.979553,1.000505,1.056781,1.155049,0.826855
2023-12-11,1.010900,1.015368,0.970935,0.994797,1.000455,1.009594


In [62]:
N = 1000
portfolios = pd.DataFrame(np.random.rand(N, df_filled.shape[1]), columns=df_filled.columns)
portfolios = (portfolios.T / portfolios.sum(axis=1)).T
portfolios

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad
0,0.117843,0.112151,0.204235,0.127553,0.132411,0.305807
1,0.169583,0.220859,0.110935,0.316113,0.027396,0.155113
2,0.212198,0.189367,0.054703,0.143076,0.111526,0.289129
3,0.097971,0.265149,0.122507,0.269757,0.130850,0.113767
4,0.230812,0.186636,0.114452,0.080368,0.071042,0.316689
...,...,...,...,...,...,...
995,0.024057,0.267446,0.295884,0.152212,0.102658,0.157742
996,0.036653,0.323195,0.268108,0.224139,0.114509,0.033396
997,0.198383,0.272818,0.320836,0.157664,0.030244,0.020055
998,0.055566,0.176941,0.134019,0.318211,0.050269,0.264993


In [63]:
outcomes = df_filled @ portfolios.T
outcomes

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-04-09,1.007141,0.990895,0.994233,1.004948,0.984784,1.015558,1.029327,1.013220,1.025885,1.010555,...,1.020530,1.032679,0.993874,0.978183,1.039194,0.999345,1.000901,0.981728,1.001876,1.024597
2022-04-10,0.920372,0.944192,0.926335,0.960212,0.922783,0.913182,0.932395,0.976726,0.955402,0.932698,...,0.981942,0.939147,0.943521,0.944406,0.960427,0.957543,0.981717,0.981061,0.921625,0.954288
2022-04-11,1.009101,1.056241,1.001865,1.050156,0.988729,1.046763,1.028839,1.032372,1.007317,1.037196,...,1.013374,1.046074,1.044618,1.026803,1.064228,1.031344,1.055518,1.043781,1.050948,1.053313
2022-04-12,1.032972,1.038241,1.040273,1.034401,1.031619,1.034460,1.057445,1.045226,1.035839,1.047297,...,1.043419,1.072366,1.023577,1.010403,1.058807,1.009965,1.015361,1.019810,1.031498,1.083129
2022-04-13,0.954267,0.960674,0.957171,0.974627,0.952284,0.946604,0.954829,0.964396,0.975829,0.945235,...,0.975151,0.948643,0.964129,0.974613,0.970197,0.974500,0.982153,0.962394,0.962724,0.951173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-09,0.963120,0.914160,0.959836,0.929366,0.976868,0.929843,0.943059,0.963240,0.977648,0.941600,...,0.981810,0.925701,0.935389,0.947320,0.917307,0.962876,0.945114,0.960448,0.912958,0.914185
2023-12-10,0.973561,0.992215,0.973181,1.011319,0.958804,0.978314,1.010214,1.031547,1.014636,0.995512,...,1.037033,1.026549,0.985276,0.968671,1.044612,0.992118,1.018515,1.006315,0.976867,1.039935
2023-12-11,0.999403,1.001874,1.005713,1.001329,1.004710,0.993243,0.999509,0.998704,1.000544,0.996766,...,1.001085,0.999335,0.998708,1.003243,0.998210,0.996541,0.996780,0.996416,1.000339,1.003632


In [64]:
means = outcomes.mean()
stds = outcomes.std()
ratios = means / stds

In [65]:
markowitz = pd.DataFrame({'mean':means, 'std':stds, 'sharpe':ratios})

In [66]:
markowitz.hvplot.scatter(y='mean',x='std',c='sharpe', cmap='RdYlGn', colorbar=True, size=10, width=500, height=500)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Scatter   [std]   (mean,sharpe)

In [67]:
ratios.argmax()

896

In [68]:
portfolios.iloc[923]

token_id
bright-token     0.089602
gitcoin          0.307247
giveth           0.004095
honey-finance    0.330611
metagame         0.170322
rad              0.098123
Name: 923, dtype: float64